In [173]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import feature_extraction
from sklearn import model_selection


In [174]:
df = pd.read_csv(r"E:\github\data science projects\2-House Prices\train.csv", )
test = pd.read_csv(r"E:\github\data science projects\2-House Prices\test.csv")

data = df.copy(deep=True)
combine_data = [data, test]


In [175]:
def data_description(file_dest: str, feature_name: str):
    try:
        import json
        dict1 = {}
        with open(file_dest, 'r') as f:
            for line in f:
                if line.strip().split(maxsplit=1):
                    s = line.strip().split(maxsplit=1)
                    dict1[s[0]] = s[1]
        print(dict1[feature_name.strip()])
    except KeyError:
       return None



file_dest = r"E:\github\data science projects\2-House Prices\data_description1.txt"
data_description(file_dest, feature_name='MSSubClass')

Identifies the type of dwelling involved in the sale.


In [176]:
def null_count(data, threshold=0.5):
    for i in data.columns:
        if data[i].isnull().mean() != 0 and data[i].isnull().mean()> threshold:
            print(f"percentage of null in   {i}:  {data[i].isnull().mean() :.3f}")
            # print(f"number of null in   {i}:  {data[i].isnull().sum() :.3f} of {data.shape[0]}",'\n')


print('null in data:')
null_count(data, 0.7)
print('\nnull in test:')
null_count(test, 0.7)

null in data:
percentage of null in   Alley:  0.938
percentage of null in   PoolQC:  0.995
percentage of null in   Fence:  0.808
percentage of null in   MiscFeature:  0.963

null in test:
percentage of null in   Alley:  0.927
percentage of null in   PoolQC:  0.998
percentage of null in   Fence:  0.801
percentage of null in   MiscFeature:  0.965


In [177]:
drop_columns = ["Alley", "PoolQC", "Fence", "MiscFeature"]

for d in combine_data:
    d.drop(drop_columns, axis=1, inplace=True)
